### *IPCC SR15 scenario assessment*

<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Global emission pathway characteristics

## Figure 3a of the *Summary for Policymakers*

This notebook extracts the emissions pathways for Figure 3a in the Summary for Policymakers
of the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

cats = specs.pop('cats')
cats_15 = specs.pop('cats_15')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

This figure only includes scenarios where the 2010 Kyoto GHG emissions are in line with the valid range as determined by the Second Assessment Report.

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
df = sr1p5.filter(category=cats_15, kyoto_ghg_2010='in range')

## Set specifications for filters and initialize data list

In [ ]:
filter_args = dict(df=df, category=cats, marker=None, join_meta=True)

In [ ]:
data = []

## Plot different emissions pathways by category

In [ ]:
writer = pd.ExcelWriter('output/spm_sr15_figure3a_data_table.xlsx')

In [ ]:
co2 = (
    df.filter(variable='Emissions|CO2')
    .convert_unit({'Mt CO2/yr': ('Gt CO2/yr', 0.001)})
    .timeseries()
)

pyam.utils.write_sheet(writer, 'Net CO2',
                       pyam.filter_by_meta(co2, **filter_args), index=True)

In [ ]:
ch4 = df.filter(variable='Emissions|CH4').timeseries()
pyam.utils.write_sheet(writer, 'CH4',
                       pyam.filter_by_meta(ch4, **filter_args), index=True)

In [ ]:
bc = df.filter(variable='Emissions|BC')
pyam.utils.write_sheet(writer, 'Black carbon',
                       pyam.filter_by_meta(bc.timeseries(), **filter_args), index=True)

In [ ]:
n2o = df.filter(variable='Emissions|N2O').timeseries()
pyam.utils.write_sheet(writer, 'N2O',
                       pyam.filter_by_meta(n2o, **filter_args), index=True)

In [ ]:
fgases = df.filter(variable='Emissions|F-Gases').timeseries()
pyam.utils.write_sheet(writer, 'F-Gases',
                       pyam.filter_by_meta(fgases, **filter_args), index=True)

In [ ]:
so2 = df.filter(variable='Emissions|Sulfur').timeseries()
pyam.utils.write_sheet(writer, 'Sulfur',
                       pyam.filter_by_meta(so2, **filter_args), index=True)

## Save timeseries data to `xlsx`

In [ ]:
writer.save()